In [ ]:
import sys
sys.path.append('/Users/emapuljak/WorkDirs/tnad/')
import random
import numpy as np
import tensorflow as tf
import math
from timeit import default_timer as timer
from tensorflow.keras.datasets import mnist

import quimb.tensor as qtn
import quimb as qu
from tnad.models import SpacedMatrixProductOperator
from sklearn.metrics import roc_curve, auc, confusion_matrix
import tnad.embeddings as e
from tnad.loss import error_logquad, error_quad

In [ ]:
def get_roc_data(normal, anomaly):
    true_val = np.concatenate((np.ones(anomaly.shape[0]), np.zeros(normal.shape[0])))
    pred_val = np.concatenate((anomaly, normal))
    fpr_loss, tpr_loss, threshold_loss = roc_curve(true_val, pred_val)
    return fpr_loss, tpr_loss

In [ ]:
def data_preprocessing(data, reshape_shape, pool_size=(2,2), strides=(2,2), padding='same'):
    # pool_size should always be (2,2)
    preprocessed=[]
    for sample in data:
        sample_tf = tf.constant(sample)
        sample_tf = tf.reshape(sample_tf, [1, 28, 28, 1])
        max_pool_2d = tf.keras.layers.MaxPooling2D(pool_size=pool_size,
           strides=strides, padding=padding)
        sample = max_pool_2d(sample_tf).numpy().reshape(reshape_shape)
        preprocessed.append(sample/255)
    return preprocessed

In [ ]:
def get_score_values(data, embedding, anomaly_score)    
    score=[]
    for sample in data:
        phi = e.embed(sample.flatten(), embedding)
        l = anomaly_score(P, phi)
        # loss
        score.append(l)
    return score

In [ ]:
model_name=''
P = qu.load_from_disk(model_name)

In [ ]:
anomaly_score = error_logquad

In [ ]:
# load data
test_size=1000
_ , (test_X, test_y) = mnist.load_data()
#anomaly
test_data=test_X[test_y!=1]
np.random.seed(123)
np.random.shuffle(test_data)
test_data=test_data[:test_size]

#normal
valid_data = test_X[test_y==1]
np.random.seed(123)
np.random.shuffle(valid_data)
valid_data=valid_data[:test_size]

In [ ]:
# preprocessing - anomalies
anomalies = data_preprocessing(test_data, (7,7), strides=(4,4))

In [ ]:
# preprocessing - normal
normal_data = data_preprocessing(valid_data, (7,7), strides=(4,4))

In [ ]:
anomalies_adscore = get_score_values(anomalies, e.trigonometric(k=1), anomaly_score)

In [ ]:
normal_adscore = get_score_values(normal_data, e.trigonometric(k=1), anomaly_score)

In [ ]:
fpr, tpr = get_roc_data(np.array(normal_adscore), np.array(anomalies_adscore))
auc_value = auc(fpr, tpr)

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.hist(np.array(anomalies_adscore), bins=100)

In [ ]:
plt.figure()
plt.hist(normal_adscore, bins=100)